# DNA Dataset Generator

This notebook parses fastq files and stores corresponding DNA sequence base calls and quality scores in shelve dictionaries for easy use. This notebook was designed for a (currently) private dataset, but can be tweaked as necessary.

In [1]:
import numpy as np
import os
import pathlib
import re
import shelve
import time

import common

In [2]:
READ_DIR = pathlib.Path("./Nachusa")
WRITE_DIR = pathlib.Path("./datasets")

In [3]:
FASTQ_REGEX = r"(?<=0000_AG_)(\d{4})-(\d{2})-(\d{2})(?=.fastq)"
BASE_CALL_MAP = { v:i for i, v in enumerate("NACGT") }

In [4]:
class FastqFile:
    def __init__(self, path, season):
        self.path = str(path)
        self.season = season
        self.samples = None
        
    def extract_sequence_id_info(self, sequence_id):
        info = {}
        
        # Split up the sequence ID information
        left, right = sequence_id.split(' ')
        left = left[1:].split(':') # Remove @ char
        right = right.split(':')
        
        info["instrument"] = left[0]
        info["run_number"] = int(left[1])
        info["flowcell_id"] = left[2]
        info["lane"] = int(left[3])
        info["tile"] = int(left[4])
        info["pos"] = map(int, left[5:])
        
        info["read_type"] = int(right[0])
        info["is_filtered"] = right[1] == 'Y'
        info["control_number"] = int(right[2])
        info["sequence_index"] = right[3]
        
        return info
        
    def load(self, shuffle=False, include_read_info=False):
        self.samples = []
        with open(self.path) as f:
            sequence_id = f.readline()
            while sequence_id:
                # Read the sample
                if include_read_info:
                    sample = self.extract_sequence_id_info(sequence_id.strip())
                else:
                    sample = {}
                sample["base_calls"] = f.readline().strip()
                f.readline() # +
                sample["quality_scores"] = f.readline().strip()
                sample["length"] = len(sample["base_calls"])
                self.samples.append(sample)

                # Next file
                sequence_id = f.readline()
    
    def free(self):
        self.samples = None
        
    def encode_sample(self, index):
        return np.array([
            [BASE_CALL_MAP[token] for token in self.samples[index]["base_calls"]],
            [(ord(token) - 33) for token in self.samples[index]["quality_scores"]]
        ])
    
    def encode_gen(self, indices):
        for i in indices:
            yield self.encode_sample(i)
        
    def encode(self, shuffle=False, split=False):
        if shuffle:
            indices = np.random.permutation(len(self.samples))
        else:
            indices = np.arange(0, len(self.samples))
        
        if split:
            if type(split) in (int, float):
                split = [split]
            split = list(map(lambda x: int(x*len(indices)), [0] + split + [1]))
            return [self.encode_gen(indices[split[i-1]:split[i]]) for i in range(1, len(split))]
        return self.encode_gen(indices)
    
    def summary(self):
        print("Total training samples:", self.train.index)
        print("Total testing samples:", self.test.index)
            
    def __enter__(self):
        self.load()
        
    def __exit__(self, type, value, traceback):
        self.free()
                
    def __str__(self):
        return self.path

## Find Files

In [5]:
seasons = [d for d in os.listdir(READ_DIR) if not d.startswith('.')]
seasons

['Fall', 'Spring']

In [6]:
files = []
for season in seasons:
    for filename in os.listdir(READ_DIR/season):
        if not filename.endswith(".fastq"):
            continue
        files.append(FastqFile(READ_DIR/season/filename, season))
        print(files[-1].path)

Nachusa/Fall/0000_AG_2016-10-07.fastq
Nachusa/Fall/0000_AG_2017-10-13.fastq
Nachusa/Spring/0000_AG_2020-05-11.fastq
Nachusa/Spring/0000_AG_2018-04-23.fastq
Nachusa/Spring/0000_AG_2017-05-02.fastq
Nachusa/Spring/0000_AG_2016-04-22.fastq
Nachusa/Spring/0000_AG_2019-05-14.fastq


## Dump All by file

In [11]:
for progress, file in enumerate(files):
    date = re.search(FASTQ_REGEX, file.path)[0]
    store = shelve.open(str(WRITE_DIR/f"samples/{file.season.lower()}_{date}"))
    with file:
        samples = file.encode(shuffle=True)
        for i, sample in enumerate(samples):
            store[str(i)] = sample
    store.close()
    print(f"\r{progress+1}/{len(files)}; File: {file.path} completed", end="")

7/7; File: Nachusa/Spring/0000_AG_2019-05-14.fastq completed

### Split Files Into Training/Testing

In [14]:
test_split = 0.2

In [15]:
files = set([os.path.splitext(f)[0] for f in os.listdir(WRITE_DIR/"samples") if re.match(r'.*\.(?:db|dat)$', f)])
files

{'fall_2016-10-07',
 'fall_2017-10-13',
 'spring_2016-04-22',
 'spring_2017-05-02',
 'spring_2018-04-23',
 'spring_2019-05-14',
 'spring_2020-05-11'}

In [17]:
for progress, name in enumerate(files):
    store = shelve.open(str(WRITE_DIR/"samples"/name))
    train = shelve.open(str(WRITE_DIR/f"{name}_train"))
    test = shelve.open(str(WRITE_DIR/f"{name}_test"))
    indices = np.random.permutation(len(store))
    partition = int(len(store)*(1 - test_split))
    for i, index in enumerate(indices[:partition]):
        train[str(i)] = store[str(index)]
    for i, index in enumerate(indices[partition:]):
        test[str(i)] = store[str(index)]
    store.close()
    train.close()
    test.close()
    print(f"\r{progress+1}/{len(files)}; File: {file.path} completed", end="")

7/7; File: Nachusa/Spring/0000_AG_2019-05-14.fastq completed